In [1]:
from collections import Counter
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Maybe we bundle all the data preprocessing in one document?

In [3]:
#read in the data
ohhla = pd.read_csv('All lyrics OHHLa.txt', sep='\n\n\n', engine='python',encoding='utf8', header = 0)

KeyboardInterrupt: 

In [ ]:
def replace_verse(x):
    if x[:5]=='Verse' or x[:6]=='Chorus' or x[:5]=='Intro' or x[0]=='[':
        return "NXTVRSE"
    if x[:4]=='Song':
        return "NXTSNG"
    else: 
        return x

In [ ]:
import re
linelist=ohhla["Artist: 40 Thevz f/ Malika"]
lines = [x for x in linelist]
lines = [x for x in lines if x[:5] != 'Album' and x[:6]!='Artist' and x[0]!='*' and x[:5]!='Typed']
lines = [replace_verse(x) for x in lines]
mini_lines = [re.sub("[^A-Za-z0-9\-\ ]", "", x.lower()) for x in lines]

In [ ]:
#Data preprocessing

In [ ]:
#split the data into the songs
count=-1
hold=[]

songlist = [None] * 100000

for item in mini_lines:
    hold.append(item)
    
    if item =="nxtsng":
        count=count+1
        songlist[count]=hold
        hold=[]
       
#remove all None
songlist = [i for i in songlist if i] 

#something still wrong here
#not all nxtvrse and nxtsng are removed 
#filters out all "nxtvrse and nxtsng markers"
filters=["nxtvrse","nxtsng"]
counter=-1
for song in songlist:
    counter=counter+1
    for item in song:
        if item == "nxtvrse" or item =="nxtsng":
            song.remove(item)
            
    
        
#remove all empyt again
songlist = [i for i in songlist if i] 

#combine the list to one string
songs=[]
for j in songlist:
    a=' '.join(j)
    songs.append(a)

In [ ]:
#Calculating the frequency of each word in each document

In [ ]:
#get a list of all words and count them
holds=[]

songs=songs[:100]


for item in songs:
    holds.append(item)

allsongs=' '.join(holds)

allwords=allsongs.split()

#count all words
allwordcount=Counter(allwords)

print("Has list of all words")


#split each song up in words
wordssongs=[]
for song in songs:
    hold=song.split()
    wordssongs.append(hold)
    
songscount=[]
for i in wordssongs:
    songscount.append(Counter(i))

print("Has list of words in songs")

In [ ]:
""" This would be faster than the cell below but it looses the words are not in the sam eorder for each document. 
Ordering them again than just brut forcing with a for loop
test=set(allwords)
counter=-1
for song in songscount:
    counter=counter+1
    key=list(song.keys())
    hold=set(key)
    miss=test.difference(hold)
    
    res = dict.fromkeys(miss, 0) 
    
    #newsong = dict(song)
    #newsong.update(res)
    song.update(res)
    songscount[counter]=song"""

In [ ]:
#calculates how often a word appears in each document

freqall=[]

for song in songscount:
    hold=[]
    
    #if word is in the song then lookup how often if not add 0
    for key in allwordcount:
        if key in song:
            val=song.get(key)
            hold.append(val)
        else:
            hold.append(0)
            
    freqall.append(hold) 

In [ ]:
#Extra calculations for inverse document frequency

In [ ]:
#calculates in how many documents each word appears
nj=[]
for key in allwordcount:
    has=0
    for song in songscount:
        if key in song:
            has=has+1
    nj.append(has)
#print(nj)

In [ ]:
#the total number of documents
N=len(songs)
print(N)

In [ ]:
#calculates the inverse document frequency for each word in each document
allwij=[]
for song in freqall:
    songfreq=[]
    for wordfreq,docfreq in zip(song,nj):
        wij=wordfreq*np.log(N/docfreq)
        songfreq.append(wij)
    allwij.append(songfreq)
    
        
    
    

In [ ]:
#Comparing the results

In [ ]:

results=cosine_similarity(freqall, freqall)
results_infreq=cosine_similarity(allwij,allwij)


In [ ]:
infreq = np.array(results_infreq)

infreq.shape=(len(results_infreq),len(results_infreq))

In [ ]:
freq=np.array(results)
freq.shape=(len(results),len(results))

In [ ]:
#plt.imshow(infreq)
#plt.colorbar()
#plt.show()

plt.imshow(freq)
plt.colorbar()
plt.show()